# Project 2 - Mc907/Mo651 - Mobile Robotics

### Student:
Luiz Eduardo Cartolano - RA: 183012

### Instructor:
Esther Luna Colombini

### Github Link:
[Project Repository](https://github.com/luizcartolano2/mc907-mobile-robotics)

### Subject of this Work:
The general objective of this work is to implement and evaluate at least 1 robot control behavior per group member.

### Goals:
1. Implement and evaluate at least 1 robot control behavior per group member (AvoidObstacle, WallFollow, Go- ToGoal) using models based on PID, Fuzzy, Neural Networks, etc;
2. Propose a behavior coordination strategy (state machine, planner, AR, subsumption, etc.)



# Code Starts Here

Import of used libraries

In [11]:
from lib import vrep
import sys, time
from src import robot as rb
from src.utils import vrep2array
import math
from time import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import skfuzzy
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from reinforcement_learning.train import *

# Defining the kinematic model of the Pionner P3DX
For this project, we are going to use the configuration of the mobile robot being characterized by the position (x,y) and the orientation in a Cartesian coordinate.

Using the follow parameters:
1. $V_R$: linear velocity of the right wheel.
2. $V_L$: linear velocity of the left wheel.
3. $W$: angular velocity of the mobile robot.
4. $X$: abscissa of the robot.
5. $Y$: intercept of the robot.
6. $X,Y$ : the actual position coordinates.
7. $\theta$: orientation of the robot.
8. $L$: the distance between the driving wheels.

The kinematic model is given by these equations [1](https://www.hindawi.com/journals/cin/2016/9548482/abs/):
<br>
\begin{align}
\frac{dX}{dt} & = \frac{V_L + V_R}{2} \cdot cos(\theta) \\
\frac{dY}{dt} & = \frac{V_L + V_R}{2} \cdot sen(\theta) \\
\frac{d \theta}{dt} & = \frac{V_L - V_R}{2} \\
\end{align}
<br>
Where ($X$,$Y$ and $\theta$) are the robot actual position and orientation angle in world reference frame. In simulation, we use the discrete form to build a model of the robot. The discrete form of the kinematic model is given by the following equations:<br>
<br>
\begin{align}
X_{k+1} & = X_k + T \cdot \frac{V_{lk} + V_{rk}}{2} \cdot cos(\theta_k + \frac{d \theta}{dt} ) \\
Y_{k+1} & = Y_k + T \cdot \frac{V_{lk} + V_{rk}}{2} \cdot sen(\theta_k + \frac{d \theta}{dt}) \\
\theta_{k+1} & = \theta_k + T \cdot \frac{V_{lk} + V_{rk}}{L} \\
\end{align}
<br>

where $X_{k+1}$ and $Y_{k+1}$ represent the position of the center axis of the mobile robot and $T$ is the sampling time.

In [2]:
class Pose:
    """
    A class used to store the robot pose.
    
    ...
    
    Attributes
    ----------
    x : double
        The x position of the robot on the map
    y : double
        The y position of the robot on the map
    orientation : double
        The angle theta of the robot on the map
        
    Methods
    -------
    The class doesn't have any methods
    """
    def __init__(self, x=None, y=None, orientation=None):
        self.x = x
        self.y = y
        self.orientation = orientation

In [1]:
class Odometry():
    """
    A class used to implement methods that allow a robot to calculate his own odometry.

    ...

    Attributes
    ----------
    robot : obj
        The robot object
    lastPose : obj Pose
        Store the robot's pose during his movement
    lastTimestamp : time
        Store the last timestamp
    left_vel : double
        Store the velocity of the left robot wheel
    right_vel : double
        Store the velocity of the right robot wheel
    delta_time : double
        Store how much time has passed
    delta_theta : double
        Store how the orientation change
    delta_space : double
        Store how the (x,y) change

    Methods
    -------
    ground_truth_updater()
        Function to update the ground truth, the real pose of the robot at the simulator
    odometry_pose_updater()
        Function to estimate the pose of the robot based on the kinematic model 
    """
    def __init__(self, robot):
        self.robot = robot
        self.lastPose = None
        self.lastTimestamp = time()
        self.left_vel = 0
        self.right_vel = 0
        self.delta_time = 0
        self.delta_theta = 0
        self.delta_space = 0
        
    def ground_truth_updater(self):
        """
            Function to update the ground truth, the real pose of the robot at the simulator
        """
        # get the (x,y,z) position of the robot at the simulator
        pose = self.robot.get_current_position()
        # get the orientation of the robot (euler angles)
        orientation = self.robot.get_current_orientation()
        
        # return an pose object (x,y,theta)
        return Pose(x=pose[0], y=pose[1], orientation=orientation[2])
    
    def odometry_pose_updater(self):
        """
            Function to estimate the pose of the robot based on the knematic model
        """
        if self.lastPose is None:
            self.lastPose = self.ground_truth_updater()
            return self.lastPose
            
        # get the actual timestamp
        time_now = time()
        # get the robot linear velocity for the left and right wheel
        left_vel, right_vel = self.robot.get_linear_velocity()
        # calculate the difference between the acutal and last timestamp
        delta_time = time_now - self.lastTimestamp
        # calculate the angle deslocation - based on the kinematic model
        delta_theta = (right_vel - left_vel) * (delta_time / self.robot.ROBOT_WIDTH)
        # calculate the distance deslocation - based on the kinematic model
        delta_space = (right_vel + left_vel) * (delta_time / 2)
        
        # auxiliary function to sum angles
        add_deltha = lambda start, delta: (((start+delta)%(2*math.pi))-(2*math.pi)) if (((start+delta)%(2*math.pi))>math.pi) else ((start+delta)%(2*math.pi))

        # calculate the new X pose
        x = self.lastPose.x + (delta_space * math.cos(add_deltha(self.lastPose.orientation, delta_theta/2)))
        # calculate the new Y pose
        y = self.lastPose.y + (delta_space * math.sin(add_deltha(self.lastPose.orientation, delta_theta/2)))
        # calculate the new Orientation pose
        theta = add_deltha(self.lastPose.orientation, delta_theta)
        
        # uptade the state of the class
        self.lastPose = Pose(x, y, theta)
        self.lastTimestamp = time_now
        self.left_vel = left_vel
        self.right_vel = right_vel
        self.delta_time = delta_time
        self.delta_theta = delta_theta
        self.delta_space = delta_space
        
        return self.lastPose

# Defining the class that controls the robot walker

For this project we are going to use two different controllers in order to make the robot avoid obstacles in the map. The first one is a classical fuzzy based system, and the second one, is a more modern approach, based on artificial intelligence, called reinforcement learning.

### Controllers:
**1. Fuzzy**
   
   Fuzzy logic is a very common technique in the Artificial Intelligence branch. It is a method that introduces the concept of partially correct and / or wrong, different from Boolean logic, in which only binary values ​​are allowed. This fact allows generalist logic in which it is not necessary to deal with all possible cases, ideal for applications with limited memory and / or time.

   A fuzzy control system is a control system based on fuzzy logic—a mathematical system that analyzes analog input values in terms of logical variables that take on continuous values between 0 and 1, in contrast to classical or digital logic, which operates on discrete values of either 1 or 0 (true or false, respectively).
   
   The input variables in a fuzzy control system are in general mapped by sets of membership functions similar to this, known as "fuzzy sets". The process of converting a crisp input value to a fuzzy value is called "fuzzification". A control system may also have various types of switch, or "ON-OFF", inputs along with its analog inputs, and such switch inputs of course will always have a truth value equal to either 1 or 0, but the scheme can deal with them as simplified fuzzy functions that happen to be either one value or another. Given "mappings" of input variables into membership functions and truth values, the microcontroller then makes decisions for what action to take, based on a set of "rules", each of the form:
~~~
    IF brake temperature IS warm AND speed IS not very fast
    THEN brake pressure IS slightly decreased.
~~~

   For this project the implemented fuzzy was very simples, and aims just to make the robot abble to avoid obstacles on his way. He uses the ultrassonic sensors for his three inputs (front, left and right distance) and outputs the linear velocity for both weels.
   
   Some fundamental concepts to understand logic are:
    
1. **Degree of Relevance:** value in the range $[0,1]$ that determines the degree to which a given element belongs to a set, allowing a gradual transition from falsehood to truth.
2. **Fuzzy Set:** A set A in X is expressed as a set of ordered pairs: $A = {{(x, \mu_A (X)) | x \in X }}$.
3. **Fuzzy Rules:** are created to evaluate the antecedent (input) and apply the result to the consequent (output). They are partially activated depending on the antecedent.
4. **Fuzzy Steps:** 
    1. *Fuzification:* stage in which subjective linguistic variables and pertinence functions are defined.
    2. *Inference:* stage at which rules are defined and evaluated.
    3. *Defuzification:* step in which the resulting regions are converted to values for the system output variable. The best known methods of defuzification are: centroid, bisector, lowest of maximum (SOM), middle of maximum (MOM), highest of maximum (LOM).

Now, we are given a more detailed explanation about the implemented system, showing how we model the inputs, outputs, system rules and the defuzzify methods. 
   
1. **Inputs and Outputs:** For this fuzzy we use three antecedents (all of them has the same shape of the one show bellow) and two consequents. The antecedents works mapping the left, right and front sensors of the ultrassonic sensors of the robot. As we can see, inputs are divide in three sets, low, medium and high distances that aims to tell the system how far he is from some object in the map. The consequents, by the other side, aims to mapping the velocity of both wheels of the robot, they are split in four velocities.
    Fuzzy Antecedent        |  Fuzzy Consequent
    :-------------------------:|:-------------------------:
    ![](images/input.png)  |  ![](images/output.png)
2. **Rules:** The system is implemented using eleven rules, that we enough to make the robot able to escape from obstacles from the map with a stable control. The rules can be describe as follow:
~~~
        left['low'] AND right['low'] AND (front['medium'] OR front['far'])),
        output_left['low'], output_right['low']

        left['low'] AND right['low'] AND front['low'],
        output_left['reverse'], output_right['low']

        left['medium'] OR left['far'] AND right['low'] AND front['low'],
        output_left['low'], output_right['high']

        left['medium'] OR left['far'] AND right['low'] AND front['medium'] OR front['far'],
        output_left['low'], output_right['high']

        left['far'] AND right['medium'] AND front['low'],
        output_left['low'], output_right['high']

        left['far'] AND right['far'] AND front['low'],
        output_left['high'], output_right['low']

        left['medium'] AND right['medium'] AND front['low'],
        output_left['high'], output_right['low']

        left['medium'] AND right['far'] AND front['low'],
        output_left['high'], output_right['low']

        left['low'] AND right['medium'] OR right['far'] AND front['low'],
        output_left['high'], output_right['low']

        left['low'] AND right['medium'] OR right['far'] AND front['medium'] OR front['far'],
        output_left['high'], output_right['low']

        left['medium'] OR left['far'] AND right['medium'] OR right['far'] AND front['medium'] OR front['far'],
        output_left['medium'], output_right['medium']
~~~
    
3. **Defuzzification:** In order to have a more conservative controller we used the minimum of the maximuns (SOM) as the defuzzification method, the consequences of this approach will be commented at the Results section.
    

In [197]:
class FuzzyControler():
    """
    A class used to implement methods that allow a robot to walk, based on a fuzzy logic controller.

    ...

    Attributes
    ----------
    forward: skfuzzy object
        Skfuzzy input object
    left: skfuzzy object
        Skfuzzy input object    
    right: skfuzzy object
        Skfuzzy input object    
    output_left: skfuzzy object
        Skfuzzy output object
    output_right: skfuzzy object
        Skfuzzy output object    
    rules: skfuzzy object
        List of rules to the fuzzy
    control: skfuzzy object
        Skfuzzy controller object
    simulator: skfuzzy object
        Skfuzzy simulator object

    Methods:
    
    -------
    create_inputs()
        Function to create skfuzzy input functions
    create_outputs()
        Function to create skfuzzy output functions        
    create_rules()
        Function to create skfuzzy rules
    create_control()
        Function to create skfuzzy controller
    show_fuzzy()
        Function to show the fuzzy rules as a graph
    create_simulator()
        Function that controls the fuzzy pipeline
    simulate()
        Function that give outputs velocity based on input distance
    """


    def __init__(self, behavior):
        self.front = None
        self.left = None
        self.right = None
        self.output_left = None
        self.output_right = None
        self.rules = []
        self.control = None
        self.simulator = None
        self.behavior = behavior
    
    def create_inputs(self):
        # set the variable universe as near, medium and far
        self.front = ctrl.Antecedent(np.arange(0, 5.01, 0.01), 'front')
        self.front['low'] = fuzz.trapmf(self.front.universe, [0, 0, 0.6, 1])
        self.front['medium'] = fuzz.trimf(self.front.universe, [0.6, 1, 1.4])
        self.front['far'] = fuzz.trapmf(self.front.universe, [1, 1.5, 5, 5])
        
        self.left = ctrl.Antecedent(np.arange(0, 5.01, 0.01), 'left')
        self.left['low'] = fuzz.trapmf(self.left.universe, [0, 0, 0.6, 1])
        self.left['medium'] = fuzz.trimf(self.left.universe, [0.6, 1, 1.4])
        self.left['far'] = fuzz.trapmf(self.left.universe, [1, 1.5, 5, 5])

        self.right = ctrl.Antecedent(np.arange(0, 5.01, 0.01), 'right')
        self.right['low'] = fuzz.trapmf(self.right.universe, [0, 0, 0.6, 1])
        self.right['medium'] = fuzz.trimf(self.right.universe, [0.6, 1, 1.4])
        self.right['far'] = fuzz.trapmf(self.right.universe, [1, 1.5, 5, 5])
        
        return
    
    def create_outputs(self):
        
        self.output_left = ctrl.Consequent(np.arange(-1, 2.01, 0.1), 'output_left')
        self.output_left['reverse'] = fuzz.trapmf(self.output_left.universe, [-1,-1, 0, 0.2])
        self.output_left['low'] = fuzz.trimf(self.output_left.universe, [0,1, 1.3])
        self.output_left['medium'] = fuzz.trimf(self.output_left.universe, [1,1.5, 1.75])
        self.output_left['high'] = fuzz.trimf(self.output_left.universe, [1.2,1.8, 2])
        self.output_left.defuzzify_method = 'som'
        
        self.output_right = ctrl.Consequent(np.arange(-1, 2.01, 0.1), 'output_right')
        self.output_right['reverse'] = fuzz.trapmf(self.output_left.universe, [-1,-1, 0, 0.2])
        self.output_right['low'] = fuzz.trimf(self.output_left.universe, [0,1, 1.3])
        self.output_right['medium'] = fuzz.trimf(self.output_left.universe, [1,1.5, 1.75])
        self.output_right['high'] = fuzz.trimf(self.output_left.universe, [1.2,1.8, 2])
        self.output_right.defuzzify_method = 'som'
        
        return
    
    def create_rules(self, front, left, right, output_left, output_right):

        rule1 = ctrl.Rule(antecedent=(left['low'] & right['low'] & (front['medium'] | front['far'])),
              consequent=(output_left['low'], output_right['low']))

        rule2 = ctrl.Rule(antecedent=(left['low'] & right['low'] & front['low']),
              consequent=(output_left['reverse'], output_right['low']))

        rule3 = ctrl.Rule(antecedent=((left['medium'] | left['far']) & right['low'] & front['low']),
              consequent=(output_left['low'], output_right['high']))

        rule4 = ctrl.Rule(antecedent=((left['medium'] | left['far']) & right['low'] & (front['medium'] | front['far'])),
              consequent=(output_left['low'], output_right['high']))

        rule5 = ctrl.Rule(antecedent=(left['far'] & right['medium'] & front['low']),
              consequent=(output_left['low'], output_right['high']))

        rule6 = ctrl.Rule(antecedent=(left['far'] & right['far'] & front['low']),
              consequent=(output_left['high'], output_right['low']))

        rule7 = ctrl.Rule(antecedent=(left['medium'] & right['medium'] & front['low']),
              consequent=(output_left['high'], output_right['low']))

        rule8 = ctrl.Rule(antecedent=(left['medium'] & right['far'] & front['low']),
              consequent=(output_left['high'], output_right['low']))

        rule9 = ctrl.Rule(antecedent=(left['low'] & (right['medium'] | right['far']) & front['low']),
              consequent=(output_left['high'], output_right['low']))

        rule10 = ctrl.Rule(antecedent=(left['low'] & (right['medium'] | right['far']) & (front['medium'] | front['far'])),
              consequent=(output_left['high'], output_right['low']))

        rule11 = ctrl.Rule(antecedent=((left['medium'] | left['far']) & (right['medium'] | right['far']) & (front['medium'] | front['far'])),
              consequent=(output_left['medium'], output_right['medium']))

        for i in range(1, 12):
            self.rules.append(eval("rule" + str(i)))
            
        return
    
    def create_control(self):
        # call function to create robot input
        self.create_inputs()
        # call function to create robot output
        self.create_outputs()
        if self.behavior == "avoid_obstacle":
            # call function to create rules
            self.create_rules(self.front, self.left, self.right, self.output_left, self.output_right)
            # create controller
            self.control = skfuzzy.control.ControlSystem(self.rules)
            
            
        return
    
    def show_fuzzy(self):         
        if self.control is None:
            raise Exception("Control not created yet!")
        else:
            self.control.view()
    
        return
    
    def create_simulator(self):
        if self.control is None:
            # crete controller if it doensn't exist
            self.create_control()

        # create simulator object
        self.simulator = ctrl.ControlSystemSimulation(self.control)
        
        return
            
    def simulate(self, input_foward=None, input_left=None, input_right=None):
        if self.simulator is None:
            # crete simulator if it doensn't exist
            self.create_simulator()
        
        # if there is no input raise exception
        if input_foward is None or input_left is None or input_right is None:
            raise Exception("Inputs can't be none")
        
        # simulte the robot linear velocity based on given inputs
        self.simulator.input['front'] = input_foward
        self.simulator.input['left'] = input_left
        self.simulator.input['right'] = input_right
        
        self.simulator.compute()
        
        return self.simulator.output['output_left'], self.simulator.output['output_right']

**2. Reinforcement Learning**
   
    

# Controls robot actions

Simple state machine that organize the robot behavior and store the informations that will be plot.

In [198]:
def state_machine(behavior="avoid_obstacle"):
    # first we create the robot and the walker object
    robot = rb.Robot()
    fuzzy = FuzzyControler(behavior=behavior)
    
    # instantiate the odometry calculator
    odometry_calculator = Odometry(robot=robot)
    
    if behavior == "follow_wall":
        raise Exception("Not implemented!")
    elif behavior == "avoid_obstacle":
        while True:
            sensors = robot.read_ultrassonic_sensors()
            front_sensors = min(sensors[3], sensors[4])
            left_sensors = min(sensors[0], sensors[1], sensors[2])
            right_sensors = min(sensors[5], sensors[6], sensors[7])
            
            #print(front_sensors, left_sensors, right_sensors)
            left_vel, right_vel = fuzzy.simulate(input_foward=front_sensors, input_left=left_sensors, input_right=right_sensors)
            
            robot.set_left_velocity(left_vel)
            robot.set_right_velocity(right_vel)
            
    else:
        raise Exception("Not implemented!")
        

# Main function - Execute the code here!
Here is a simple signal handler implement in order to make the simulator execution last for a given time period.

In [199]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

try:
    ground_truth = []
    odometry = []
    lines = []
    corners = []
    points_kmeans = []
    with time_limit(90):
        state_machine()
except TimeoutException as e:
    print("Timed out!")

Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.


KeyboardInterrupt: 

# Some Results
In order to show how the implemented code works we are going to do and discuss three experiments.

## Conclusions And Final Observations

Once the project is done, it is clear that there are a number of improvements to be implemented. For the kinematic model, more sophisticated solutions, as visual odometry, visual slam, or even implement a kalman filter for odometry would give us much better results. For the mapping, even though we have obtained useful results, the K-Means suffers with to much noise, separating us from high quality results. In order to achieve better mapping results, we could implement feature extraction from images (like we were trying, but we failed), or state of art algorithms, like the Bresenham's line drawing algorithm, or the raincast, among other famous ones.

Personally speaking, I believe the results were far below expectations. The learning curve for understanding the robot pipeline was longer than expected. Also, I believe a lot of time was spent on minor issues for this project, such as the implementation of the fuzzy controller. For future projects, better programming and preparation is needed to achieve better results.